# Subqueries

Though not strictly a JOIN, a subquery needs to be discussed here as they are means of utilising queries and tables in a manner that can be perceived as similar to a JOIN.

#### What is a Subquery?

A subquery is the name given to a query which is nested inside another query.

A subquery can be placed in many areas of a SQL statement although the most common forms are as Derived Tables (which we discuss shortly) and as the source of IN lists.

As an example, we may wish to write a query which reports on all sales made by members of the Telesales team.  We may currently have 20 employees within that team which is a large, but not unmanageable amount of IDs to manually type into an IN list within the WHERE clause of our query:

<code>WHERE employeeID IN (1, 4, 6, 22, 42, 25, 87 ..... )</code>

However, now let's imagine that we are expanding our operations rapidly and we now have 200 employees in our Telesales team and turnover is high so that we are hiring and firing new sales people each week.  Suddenly our manual list becomes impossible to safely maintain without too high a risk of mistake, outdated data, and error.

Wouldn't it be nice if we could write something akin to the following?

<code>WHERE employeeID IN<br/>
(<br/>
    SELECT employeeID<br/>
    FROM myTable<br/>
    WHERE teamName = 'Telesales'<br/>
)</code>

Well that's exactly what a subquery allows us to do.  As mentioned above, they are a query nested inside another query.

This provides us with the correct list of employeeIDs every single time the query is run and without any risk of error.

#### Why use a subquery?

If you think about the above, we can actually write that same statement as a JOIN.

Generally speaking all subqueries could be written as a JOIN instead of as a subquery (this is not always the case, but generally speaking it is), therefore why do we bother?

It comes down to the way in which you think about your queries, the logic involved, and how you yourself feel comfortable coding.  Some people will use nothing but subqueries, others none, whereas many will be somewhere in between.

Luckily SQL Server is a very clever piece of software and re-writes everything we pass it into a logical flow that it can understand and therefore using a subquery or a join rarely makes a difference to how SQL Server processes the query and therefore performs.

Let's look at some examples using the same employee and sales data that we have seen throughout our demos so far.

#### Subquery in SELECT

We are not limited to the IN clause but can place these subqueries in several places, one of which is the SELECT line.

We shall presume that, for some reason, we wish to see the name of the CEO (the employee with managerID NULL) next to every employee.

One way to imagine this is by use of the CROSS JOIN in which we place every record of our JOIN table against every record of our FROM table.  Therefore if we only return one record in the JOIN (our CEO) then this will be placed against every record in our FROM table.

In code this looks as follows:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		concat_ws(' ', ceo.firstName, ceo.lastName) as CEOFullName
from JupyterDatabase.hr.employee as e
cross join JupyterDatabase.hr.employee as ceo
where ceo.managerID is null;

However, now let's think of this in terms of a subquery.

We wish each row to show the CEO's name.  Therefore we need each record returned to show the result of the following query:

<code>select concat_ws(' ', firstName, lastName) as CEOFullName<br/>
from JupyterDatabase.hr.employee<br/>
where managerID is null;</code>

So why can't we do exactly that?  With a subquery, we can:

In [ ]:
select employeeID, firstName, lastName,
		(
			select concat_ws(' ', firstName, lastName)
			from JupyterDatabase.hr.employee
			where managerID is null
		) as CEOFullName
from JupyterDatabase.hr.employee;

#### Subquery in WHERE

As per our walk-through example at the start of this section we can also place a subquery in the WHERE clause of our query, satisfying an IN clause in order to provide a self-updating list of values to our query.

Let's do that now by obtaining a TotalSales figure for any employee with jobTitleID of 25.

We COULD, at this point, use a manual list for our IN clause, but even with our small demo table we have far too many for this to be practical:

In [ ]:
select count(*)
from JupyterDatabase.hr.employee
where jobTitleID = 25;

So let's replace our manual list with a subquery:

In [ ]:
select SUM(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
where salesPersonID in
(
	select employeeID
	from JupyterDatabase.hr.employee
	where jobTitleID = 25
);

Our query now automatically adapts to any employees who are added to or removed from the team.

Again, this could be done through a JOIN instead depending on how you think or prefer to write your queries:

In [ ]:
select SUM(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader as s
join JupyterDatabase.hr.employee as e
on s.salesPersonID = e.employeeID
where e.jobTitleID = 25;

Therefore, as you can see, it all depends on how you think about your queries in your head as to how you may wish to code. It simply helps to know that either way is valid.

The last piece of syntax we need to consider is the Derived Table, which is a variant of a Subquery and will be discussed next.